# Try CoreVital

Run this notebook in Google Colab to try CoreVital in under 2 minutes. No GPU required.

CoreVital instruments LLM inference to capture hidden states, attention patterns, and logits,
then computes health flags, risk scores, and human-readable narratives.

In [ ]:
# Install CoreVital from GitHub (force-reinstall ensures the correct branch code)
!pip uninstall -y CoreVital 2>/dev/null
!pip install --force-reinstall --no-cache-dir "git+https://github.com/Joe-b-20/CoreVital.git@release/showcase"

In [ ]:
# Verify installation
import CoreVital
print(f"CoreVital {CoreVital.__version__} installed at {CoreVital.__file__}")
print(f"CoreVitalMonitor available: {hasattr(CoreVital, 'CoreVitalMonitor')}")

In [ ]:
from CoreVital import CoreVitalMonitor

monitor = CoreVitalMonitor(capture_mode="summary", max_new_tokens=20)

# Change this prompt to anything you want
PROMPT = "Explain why the sky is blue in one sentence."

monitor.run("gpt2", PROMPT)

print("Generated:", monitor.get_report().generated.output_text)
print()
print("Risk score:", monitor.get_risk_score())
print("Health flags:", monitor.get_health_flags())
print("Narrative:", monitor.get_summary().get("narrative", {}).get("summary", ""))

In [ ]:
# Inspect the full report
report = monitor.get_report()

print(f"Model: {report.model.hf_id} ({report.model.num_layers} layers)")
print(f"Prompt tokens: {report.summary.prompt_tokens}")
print(f"Generated tokens: {report.summary.generated_tokens}")
print(f"Wall time: {report.summary.elapsed_ms:.0f} ms")
print()

# Per-step entropy curve
for step in report.timeline:
    ent = step.logits_summary.entropy if step.logits_summary else None
    tok = step.token.token_text if step.token else "?"
    print(f"  step {step.step_index:2d}  token={tok:<15s}  entropy={ent:.2f}" if ent else f"  step {step.step_index:2d}  token={tok}")

In [ ]:
# Health-aware decoding: should the system intervene?
monitor_strict = CoreVitalMonitor(
    capture_mode="summary",
    max_new_tokens=20,
    intervene_on_risk_above=0.7,
)
monitor_strict.run("gpt2", "The meaning of life is")

print("Risk:", monitor_strict.get_risk_score())
print("Should intervene:", monitor_strict.should_intervene())
print("Output:", monitor_strict.get_report().generated.output_text)

## Next steps

- **Dashboard:** Install locally and run `streamlit run dashboard.py` to visualize reports
- **Production models:** Try with Llama, Mistral, or Qwen (requires more memory or GPU)
- **Docs:** See the [README](https://github.com/Joe-b-20/CoreVital) for CLI usage, configuration, and architecture